In [1]:
import os
# import lxml
from lxml import etree
import pandas as pd
import numpy as np

In [2]:
import sys

In [3]:
from nltk.classify.scikitlearn import SklearnClassifier

In [4]:
import fasttext

In [5]:
from collections import defaultdict

In [6]:
from sklearn.metrics import classification_report

In [7]:
from scipy.sparse import csc_matrix

In [8]:
import os

import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
from sklearn.preprocessing import Normalizer

In [11]:
from nltk.util import ngrams

In [12]:
cl = ['статья','интервью','комментарий']

In [13]:
f = open('./out.csv', 'r')

file_hash = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    filename = os.path.basename(path)
    if class_name in cl:
        file_hash[filename] = class_name


In [14]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

In [15]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text

In [16]:
sent = []
for tree, class_text in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    all_p = tree.xpath('//p')
    joined = '\n'.join([p.text for p in all_p if p.text is not None])
    sent.append((joined, class_text))

In [17]:
sent = sent[:800]

In [18]:
only_texts = [text[0] for text in sent]
classes = [text[1] for text in sent]

In [19]:
d = defaultdict(int)
for word in classes:
    d[word] += 1

In [20]:
d

defaultdict(int, {'интервью': 197, 'комментарий': 230, 'статья': 373})

In [21]:
bigrams = set()
num = 1
for text in sent:
    text_clear = [word.lower().strip('.,!-?»«') for word in text[0].split()]
    text_2grams = list(ngrams(list(text_clear), 2))
    for i in text_2grams:
        bigrams.add(i)
        
bigrams = list(bigrams)

In [22]:
str_bigrams = [' '.join(bi) for bi in bigrams]

In [23]:
index = range(len(sent))

In [24]:
bigrams_per_sent = []
for i, text in enumerate(sent):
    text_clear = [word.lower().strip('.,!-?»«') for word in text[0].split()]  # биграммы из текста
    text_2grams = list(ngrams(list(text_clear), 2)) # биграммы из текста
    str_2grams = set([' '.join(bi) for bi in text_2grams])
    bigrams_per_sent.append(str_2grams)

In [25]:
all_vectors = []
for i, text in enumerate(sent):
    vector = []
    for bi in str_bigrams:
        if bi in bigrams_per_sent[i]:
            vector.append(1)
        else:
            vector.append(0)
    all_vectors.append(vector)

print(len(all_vectors))

800


In [26]:
X_train, X_test, y_train, y_test = train_test_split(all_vectors, classes, test_size=0.3)

In [27]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   интервью       0.50      0.05      0.10        56
комментарий       0.56      0.81      0.66        74
     статья       0.61      0.70      0.65       110

avg / total       0.57      0.58      0.52       240



In [28]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print(classification_report(y_test, y_pred_svc))

             precision    recall  f1-score   support

   интервью       0.00      0.00      0.00        56
комментарий       0.00      0.00      0.00        74
     статья       0.46      1.00      0.63       110

avg / total       0.21      0.46      0.29       240



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
regr = LogisticRegression()
regr.fit(X_train, y_train)
y_pred_regr = regr.predict(X_test)
print(classification_report(y_test, y_pred_regr))

             precision    recall  f1-score   support

   интервью       0.42      0.09      0.15        56
комментарий       0.72      0.70      0.71        74
     статья       0.61      0.86      0.71       110

avg / total       0.60      0.63      0.58       240



In [31]:
y_pred_rest = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, y_pred_rest))

             precision    recall  f1-score   support

   интервью       0.53      0.18      0.27        56
комментарий       0.69      0.84      0.76        74
     статья       0.66      0.78      0.71       110

avg / total       0.64      0.66      0.62       240



Gradient Descent

In [32]:
clf_grad = linear_model.SGDClassifier()
clf_grad.fit(X_train, y_train)
y_pred_grad = clf_grad.predict(X_test)
print(classification_report(y_test, y_pred_grad))

             precision    recall  f1-score   support

   интервью       0.55      0.41      0.47        56
комментарий       0.71      0.68      0.69        74
     статья       0.68      0.79      0.73       110

avg / total       0.66      0.67      0.66       240



In [33]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

clf_bayes = MultinomialNB()
clf_bayes.fit(X_train_norm, y_train)
y_pred_bayes = clf_bayes.predict(X_test_norm)

print(classification_report(y_test, y_pred_bayes))

             precision    recall  f1-score   support

   интервью       0.38      0.73      0.50        56
комментарий       0.45      0.77      0.56        74
     статья       0.33      0.01      0.02       110

avg / total       0.38      0.41      0.30       240



In [34]:
clf_tree = DecisionTreeClassifier(random_state=0)
clf_tree.fit(X_train, y_train)
y_pred_tree = clf_tree.predict(X_test)
print(classification_report(y_test, y_pred_tree))

             precision    recall  f1-score   support

   интервью       0.43      0.39      0.41        56
комментарий       0.67      0.61      0.64        74
     статья       0.66      0.74      0.70       110

avg / total       0.61      0.62      0.61       240



# N = 3

In [35]:
trigrams = set()
num = 1
for text in sent:
    text_clear = [word.lower().strip('.,!-?»«') for word in text[0].split()]
    text_3grams = list(ngrams(list(text_clear), 3))
    for i in text_3grams:
        trigrams.add(i)
        
trigrams = list(trigrams)

In [48]:
str_trigrams = [' '.join(tri) for tri in trigrams]
index = range(len(sent))

In [37]:
trigrams_per_sent = []
for i, text in enumerate(sent):
    text_clear = [word.lower().strip('.,!-?»«') for word in text[0].split()]  # биграммы из текста
    text_3grams = list(ngrams(list(text_clear), 3)) # биграммы из текста
    str_3grams = set([' '.join(tri) for tri in text_3grams])
    trigrams_per_sent.append(str_3grams)

In [38]:
all_vectors = []
for i, text in enumerate(sent):
    vector = []
    for tri in str_trigrams:
        if tri in trigrams_per_sent[i]:
            vector.append(1)
        else:
            vector.append(0)
    all_vectors.append(vector)

print(len(all_vectors))

800


In [40]:
X_train, X_test, y_train, y_test = train_test_split(all_vectors, classes, test_size=0.3)

In [41]:
clf_grad = linear_model.SGDClassifier()
clf_grad.fit(X_train, y_train)
y_pred_grad = clf_grad.predict(X_test)

'             precision    recall  f1-score   support\n\n   интервью       0.40      0.18      0.24        57\nкомментарий       0.71      0.63      0.67        73\n     статья       0.63      0.85      0.72       110\n\navg / total       0.60      0.62      0.59       240\n'

In [42]:
print(classification_report(y_test, y_pred_grad))

             precision    recall  f1-score   support

   интервью       0.40      0.18      0.24        57
комментарий       0.71      0.63      0.67        73
     статья       0.63      0.85      0.72       110

avg / total       0.60      0.62      0.59       240



In [43]:
clf_for = RandomForestClassifier()
clf_for.fit(X_train, y_train)
y_pred_for = clf_for.predict(X_test)
print(classification_report(y_test, y_pred_for))

             precision    recall  f1-score   support

   интервью       0.00      0.00      0.00        57
комментарий       0.94      0.21      0.34        73
     статья       0.49      1.00      0.66       110

avg / total       0.51      0.52      0.40       240



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [44]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print(classification_report(y_test, y_pred_svc))

             precision    recall  f1-score   support

   интервью       0.00      0.00      0.00        57
комментарий       0.00      0.00      0.00        73
     статья       0.46      1.00      0.63       110

avg / total       0.21      0.46      0.29       240



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
y_pred_rest = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, y_pred_rest))

             precision    recall  f1-score   support

   интервью       0.00      0.00      0.00        57
комментарий       0.35      0.95      0.51        73
     статья       0.63      0.25      0.35       110

avg / total       0.39      0.40      0.32       240



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [46]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

clf_bayes = MultinomialNB()
clf_bayes.fit(X_train_norm, y_train)
y_pred_bayes = clf_bayes.predict(X_test_norm)

print(classification_report(y_test, y_pred_bayes))

             precision    recall  f1-score   support

   интервью       0.00      0.00      0.00        57
комментарий       0.30      1.00      0.47        73
     статья       0.00      0.00      0.00       110

avg / total       0.09      0.30      0.14       240



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [49]:
clf_tree = DecisionTreeClassifier(random_state=0)
clf_tree.fit(X_train, y_train)
y_pred_tree = clf_tree.predict(X_test)
print(classification_report(y_test, y_pred_tree))

             precision    recall  f1-score   support

   интервью       0.35      0.19      0.25        57
комментарий       0.55      0.79      0.65        73
     статья       0.65      0.61      0.63       110

avg / total       0.55      0.57      0.54       240

